In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler

# Load features and labels
x_1gram = pd.read_csv('C:\\Users\\as998\\OneDrive\\Documents\\1gram_features.csv').values
x_2gram = pd.read_csv('C:\\Users\\as998\\OneDrive\\Documents\\2gram_features.csv').values
y = pd.read_csv('C:\\Users\\as998\\OneDrive\\Documents\\labels.csv')['apt'].values

# Apply RandomOverSampler to balance the dataset
ros = RandomOverSampler(random_state=42)
x_1gram_resampled, y_resampled = ros.fit_resample(x_1gram, y)
x_2gram_resampled, _ = ros.fit_resample(x_2gram, y)

# Split dataset for both 1-gram and 2-gram matrices
x_train_1gram, x_test_1gram, y_train, y_test = train_test_split(
    x_1gram_resampled, y_resampled, test_size=0.3, random_state=42)
x_train_2gram, x_test_2gram, _, _ = train_test_split(
    x_2gram_resampled, y_resampled, test_size=0.3, random_state=42)

# Define classifiers with hyperparameter grids
classifiers = {
    'svm': {
        'model': SVC(),
        'params': {
            'svm__C': [1, 10, 100],
            'svm__kernel': ['linear', 'rbf'],
            'svm__class_weight': ['balanced']
        }
    },
    'kneighborsclassifier': {
        'model': KNeighborsClassifier(),
        'params': {
            'kneighborsclassifier__n_neighbors': [3],
            'kneighborsclassifier__weights': ['uniform', 'distance']
        }
    },
    'decisiontreeclassifier': {
        'model': DecisionTreeClassifier(),
        'params': {
            'decisiontreeclassifier__max_depth': [5, 10, 15, None],
            'decisiontreeclassifier__class_weight': ['balanced', None]
        }
    }
}

# Function to evaluate model performance and confusion matrix
def evaluate_model(model, x_test, y_test, model_name):
    y_pred = model.predict(x_test)
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred) * 100
    recall = recall_score(y_test, y_pred, average='macro', zero_division=1)
    precision = precision_score(y_test, y_pred, average='macro', zero_division=1)
    f1 = f1_score(y_test, y_pred, average='macro')
    
    # Print performance metrics
    print(f"Performance on {model_name} test set:")
    print(f"Accuracy: {accuracy:.2f}% | Recall: {recall:.2f} | Precision: {precision:.2f} | F1 Score: {f1:.2f}")
    
    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print(f"\nConfusion Matrix for {model_name}:\n{cm}")
    
    # Calculate TP, TN, FP, FN
    tp = cm.diagonal().sum()
    fn = cm.sum(axis=1) - cm.diagonal()
    fp = cm.sum(axis=0) - cm.diagonal()
    tn = cm.sum() - (fp + fn + tp)
    
    # Print TP, TN, FP, FN
    print(f"Metrics for {model_name}:")
    print(f"True Positive (TP): {tp}")
    print(f"True Negative (TN): {tn.sum()}")
    print(f"False Positive (FP): {fp.sum()}")
    print(f"False Negative (FN): {fn.sum()}")
    
    return accuracy, recall, precision, f1

# Define KFold cross-validation strategy
cv_strategy = KFold(n_splits=5)

# Pipeline for scaling and grid search
for name, clf_info in classifiers.items():
    pipeline_1gram = Pipeline([
        ('scaler', StandardScaler()),
        (name, clf_info['model'])
    ])
    pipeline_2gram = Pipeline([
        ('scaler', StandardScaler()),
        (name, clf_info['model'])
    ])
    
    grid_search_1gram = GridSearchCV(pipeline_1gram, clf_info['params'], cv=cv_strategy, scoring='accuracy', n_jobs=-1)
    grid_search_2gram = GridSearchCV(pipeline_2gram, clf_info['params'], cv=cv_strategy, scoring='accuracy', n_jobs=-1)
    
    # 1-Gram Processing
    print(f"\033[1m--- {name.upper()} with 1-Gram Processing ---\033[0m")
    grid_search_1gram.fit(x_train_1gram, y_train)
    best_model_1gram = grid_search_1gram.best_estimator_
    print(f"Best parameters for {name.upper()} (1-Gram): {grid_search_1gram.best_params_}")
    evaluate_model(best_model_1gram, x_test_1gram, y_test, name.upper() + " (1-Gram)")

    # 2-Gram Processing
    print(f"\033[1m--- {name.upper()} with 2-Gram Processing ---\033[0m")
    grid_search_2gram.fit(x_train_2gram, y_train)
    best_model_2gram = grid_search_2gram.best_estimator_
    print(f"Best parameters for {name.upper()} (2-Gram): {grid_search_2gram.best_params_}")
    evaluate_model(best_model_2gram, x_test_2gram, y_test, name.upper() + " (2-Gram)")


--- SVM with 1-Gram Processing ---
Best parameters for SVM (1-Gram): {'svm__C': 1, 'svm__class_weight': 'balanced', 'svm__kernel': 'linear'}
Performance on SVM (1-Gram) test set:
Accuracy: 63.51% | Recall: 0.67 | Precision: 0.79 | F1 Score: 0.57

Confusion Matrix for SVM (1-Gram):
[[3 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 3 ... 0 0 0]
 ...
 [0 0 0 ... 2 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]]
Metrics for SVM (1-Gram):
True Positive (TP): 47
True Negative (TN): 864
False Positive (FP): 27
False Negative (FN): 27
--- SVM with 2-Gram Processing ---
Best parameters for SVM (2-Gram): {'svm__C': 100, 'svm__class_weight': 'balanced', 'svm__kernel': 'rbf'}
Performance on SVM (2-Gram) test set:
Accuracy: 64.86% | Recall: 0.68 | Precision: 0.83 | F1 Score: 0.60

Confusion Matrix for SVM (2-Gram):
[[3 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 3 ... 0 0 0]
 ...
 [0 0 0 ... 2 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]]
Metrics for SVM (2-Gram):
True Positive (TP): 48
True Negative (TN): 832
Fal